In [ ]:
import requests, pickle, pandas, datetime, os, shutil, time, functools, json, hashlib, xlrd

from openpyxl.styles import PatternFill
baseurl=os.getenv('BASE_URL')
api_person = baseurl+"/api/Person"
api_police = baseurl+"/api/Polis"
ecp_searchData = baseurl+"/?c=Search&m=searchData"
#cookie_file=Path("cookies.txt")
cookie_file="cookies.txt"
year=2025
month=10

def month_range(year,month):
    today=datetime.date.today()
    range_start=datetime.date(year,month,1).strftime("%d.%m.%Y")
    range_end=(datetime.date(year,month+1,1)-datetime.timedelta(days=1)).strftime("%d.%m.%Y")
    return range_start+" - " + range_end

def find_seven(year, month, path):
    file_list = os.listdir(path)
    for file in file_list:
        if (str(year)+'-'+str(month).zfill(2)) in file:
            sevens_filename=file
    return path+'\\'+sevens_filename

def rate_limit(minutes=30, cache_dir=".cache"):
    """
    Decorator that prevents a function from being called more than once within specified minutes
    with the same arguments. Stores both the last call time and the result in persistent files.
    Returns the cached result if called too soon with the same arguments.
    
    Args:
        minutes (int): Minimum minutes between function calls
        cache_dir (str): Directory to store cache files
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if not os.path.exists(cache_dir):
                os.makedirs(cache_dir)
            key_parts = [func.__module__, func.__name__]
            for arg in args:
                key_parts.append(str(arg))
            for k in sorted(kwargs.keys()):
                key_parts.append(f"{k}:{kwargs[k]}")
            args_hash = hashlib.md5(str(key_parts).encode()).hexdigest()
            index_file = os.path.join(cache_dir, f"{func.__name__}_index.json")
            result_file = os.path.join(cache_dir, f"{func.__name__}_{args_hash}.pkl")
            index = {}
            if os.path.exists(index_file):
                try:
                    with open(index_file, 'r') as f:
                        index = json.load(f)
                except (json.JSONDecodeError, IOError):
                    index = {}
            
            current_time = time.time()
            if args_hash in index:
                last_call_time = index[args_hash]["timestamp"]
                time_diff = current_time - last_call_time
                if time_diff < (minutes * 60):
                    print(f"Using cached result. Last called {time_diff/60:.1f} minutes ago with same arguments.")
                    try:
                        with open(result_file, 'rb') as f:
                            return pickle.load(f)
                    except (IOError, pickle.PickleError):
                        pass
            result = func(*args, **kwargs)
            index[args_hash] = {
                "timestamp": current_time,
                "args": str(args),
                "kwargs": str(kwargs)
            }
            with open(index_file, 'w') as f:
                json.dump(index, f)
            with open(result_file, 'wb') as f:
                pickle.dump(result, f)
                
            return result
        return wrapper
    return decorator

@rate_limit(minutes=30)
def ecp_find_kss_in_daterange(date_range, in_or_out, kss_only):
    session = requests.session()
    print("Logining")
    headers = {
            "Login": os.getenv('ECP_USERNAME_1'),
            "Password": os.getenv('ECP_PASSWORD_1')
    }
    session.request("GET", f"{baseurl}/api/user/login", headers=headers)
    
    payload = {'PersonPeriodicType_id': '1', 'SearchFormType': 'EvnPS', 'Date_Type':'1', 'limit':'10000'}
    if in_or_out=='in':
        payload['EvnPS_setDate_Range']=date_range
        print('Ищем поступивших за период ' + date_range)
    else:
        payload['EvnPS_disDate_Range']=date_range
        print('Ищем выбывших за период ' + date_range)
    if kss_only:
        payload['LpuUnitType_did']=1
        print('только случаи КСС')
    response = session.request("GET", ecp_searchData, params=payload)
    return response.json()

def write_data_to_sevens(sevens_filename, div_name):
    print('Start write_data_to_sevens:', div_name, end='. ')
    try:
        backup_filename = sevens_filename + '.backup'
        import shutil
        shutil.copy2(sevens_filename, backup_filename)
        from openpyxl import load_workbook
        from openpyxl.styles import PatternFill
        try:
            workbook = load_workbook(sevens_filename)
            sheets = workbook.sheetnames
        except Exception as e:
            print(f"Error loading with openpyxl: {e}")
            wb = xlrd.open_workbook(sevens_filename)
            sheets = wb.sheet_names()
        good_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
        neutral_fill = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")
        bad_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
        from openpyxl import Workbook
        new_workbook = Workbook()
        if "Sheet" in new_workbook.sheetnames:
            new_workbook.remove(new_workbook["Sheet"])
        print('Парсим', sevens_filename)
        for sheet_name in sheets:
            sheet_data = pandas.read_excel(sevens_filename, sheet_name=sheet_name)
            if sheet_name not in new_workbook.sheetnames:
                new_workbook.create_sheet(sheet_name)
            new_sheet = new_workbook[sheet_name]
            for row_idx, row in sheet_data.iterrows():
                for col_idx, value in enumerate(row):
                    new_sheet.cell(row=row_idx+1, column=col_idx+1, value=value)
            try:
                sheet_day_cell_value = pandas.read_excel(sevens_filename, sheet_name=sheet_name, skiprows=2, nrows=101, header=None).iloc[0][3]
                if type(sheet_day_cell_value) == datetime.datetime:
                    sheet_day = (sheet_day_cell_value - datetime.timedelta(days=1)).strftime("%d.%m.%Y")
                else:
                    sheet_day = (datetime.datetime.strptime(sheet_day_cell_value, "%d.%m.%Y") - (datetime.timedelta(days=1))).strftime("%d.%m.%Y")
                discharged = pandas.read_excel(sevens_filename, sheet_name=sheet_name, skiprows=8, usecols=[2]).dropna()
                transfered = pandas.read_excel(sevens_filename, sheet_name=sheet_name, skiprows=8, usecols=[6]).dropna()
                transfered.rename(columns={'Перевод\nв др. ЛПУ': 'Выписано'}, inplace=True)
                discharged = pandas.concat([discharged, transfered])
                discharged['ДатаВыписки'] = sheet_day
                discharged['Отделение'] = div_name
                if len(discharged) > 0:
                    sevens_total.append(discharged.to_dict('records'))
                for idx, value in enumerate(discharged.sort_values(by='Выписано').iloc[:, 0]):
                    new_sheet.cell(row=10+idx, column=11, value=value)
                new_sheet.cell(column=11, row=9, value=("Выписанные по семёркам" if len(discharged) != 0 else "Выписки в семёрках нет"))
                alive = pandas.DataFrame([pacient for pacient in pacients if pacient['ДатаВыписки'] == sheet_day and pacient['Отделение'] == div_name and pacient['Исход'] != '105'])
                if len(alive) > 0:
                    for idx, value in enumerate(alive.iloc[:, 0].sort_values()):
                        new_sheet.cell(row=10+idx, column=12, value=value)
                new_sheet.cell(column=12, row=9, value=("Выписанные в ЕЦП" if len(alive) != 0 else "Выписки в ЕЦП нет"))
                dead = pandas.DataFrame([pacient for pacient in pacients if pacient['ДатаВыписки'] == sheet_day and pacient['Отделение'] == div_name and pacient['Исход'] == '105'])
                if len(dead) > 0:
                    for idx, value in enumerate(dead.iloc[:, 0].sort_values()):
                        new_sheet.cell(row=10+idx, column=13, value=value)
                new_sheet.cell(column=13, row=9, value=("Трупы в ЕЦП" if len(dead) != 0 else "Трупов в ЕЦП нет"))
                if len(alive) == len(discharged) and len(alive) > 0:
                    new_sheet['E7'].fill = good_fill
                elif len(alive) == len(discharged) and len(alive) == 0:
                    new_sheet['E7'].fill = neutral_fill
                else:
                    new_sheet['E7'].fill = bad_fill
                new_sheet['E8'] = len(alive)
            except Exception as e:
                print(f"Error processing sheet {sheet_name}: {e}")
        temp_filename = sevens_filename + '.new'
        new_workbook.save(temp_filename)
        import os
        os.remove(sevens_filename)
        os.rename(temp_filename, sevens_filename)
        
    except Exception as e:
        print(f"Error in write_data_to_sevens: {e}")
        try:
            if os.path.exists(backup_filename):
                shutil.copy2(backup_filename, sevens_filename)
                print("Restored from backup")
        except:
            pass

In [ ]:

#------------------initialisation--------------------

sevens_total=[]
sevens_count,ecp_count,ecp_x=0,0,0

div_list=[{'Отделение':'Инфекционное детское №1', 'Файл':find_seven(year,month,r'S:\06 - Отделения\1 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},
          {'Отделение':'Инфекционное №2', 'Файл':find_seven(year,month,r'S:\06 - Отделения\2 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},
          {'Отделение':'Инфекционное №3', 'Файл':find_seven(year,month,r'S:\06 - Отделения\3 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},
          {'Отделение':'Инфекционное №4', 'Файл':find_seven(year,month,r'S:\06 - Отделения\4 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},]


#------------------here-we-are-ready--------------------

kss_in_range_response=ecp_find_kss_in_daterange(month_range(year=year,month=month),'out',1)
pacients=[]
ecp_x=0
for pacient in kss_in_range_response['data']:
    if pacient['LeaveType_Code']!='104': #игнорируем переведённых на другой профиль коек
        pacients.append({'ФИО':pacient['Person_Surname']+" "+pacient['Person_Firname'],
                         'ДатаВыписки':pacient['EvnPS_disDate'],
                         'Отделение':pacient['LpuSection_Name'],
                         'Исход':pacient['LeaveType_Code']})
        if pacient['LeaveType_Code']!='105': #не труп
            ecp_count+=1
        else: #труп
            ecp_x+=1
        for div in div_list:
            if div['Отделение']==pacient['LpuSection_Name'] and pacient['LeaveType_Code']!='105':
                div['ВыписаноЕЦП']+=1

print('пациентов в ЕЦП ' + str(len(pacients)))

for div in div_list:
    shutil.copy2(div['Файл'],'.')
    write_data_to_sevens(div['Файл'].split('\\')[-1],div['Отделение'])

sevens_count=0
for day in sevens_total:
    for p in day:
        sevens_count+=1
        for div in div_list:
            if div['Отделение']==p['Отделение']:
                div['Выписано7']+=1

print(datetime.datetime.now())
print('ВСЕГО: по семёркам', sevens_count, 'выписано в ЕЦП', ecp_count, ', не выписано в ЕЦП', (sevens_count-(ecp_count)), ', трупы',ecp_x)

for div in div_list:
    percentage = 0
    if div['Выписано7'] > 0:
        percentage = (div['Выписано7'] - div['ВыписаноЕЦП'])/div['Выписано7']*100
    
    print (div['Отделение'][-1:]+': 7-ки', str(div['Выписано7'])+
           ', ЕЦП',str(div['ВыписаноЕЦП'])+
           ', невыписано',div['Выписано7'] - div['ВыписаноЕЦП'],
           "%.1f" % percentage+'%'+
           ', дата заполнения:',time.ctime(os.path.getmtime(div['Файл']))[4:-8]
          )

counter=0
for p in pacients:
    if p['Исход'] not in ['101','102','105']:
        
        print(counter, p['Исход'], p['ФИО'])
        counter+=1

Logining
Ищем выбывших за период 01.10.2025 - 31.10.2025
только случаи КСС
пациентов в ЕЦП 524
Start write_data_to_sevens: Инфекционное детское №1. Парсим семёрки 2025-10 Iотд.xlsx
Start write_data_to_sevens: Инфекционное №2. Парсим семёрки 2025-10 IIотд.xlsx
Start write_data_to_sevens: Инфекционное №3. Парсим семёрки 2025-10 IIIотд.xlsx
Start write_data_to_sevens: Инфекционное №4. Парсим семёрки 2025-10 IVотд.xlsx
Error processing sheet Лист1: single positional indexer is out-of-bounds
2025-11-01 09:42:13.608509
ВСЕГО: по семёркам 524 выписано в ЕЦП 487 , не выписано в ЕЦП 37 , трупы 37
1: 7-ки 209, ЕЦП 175, невыписано 34 16.3%, дата заполнения: Nov  1 07:42
2: 7-ки 119, ЕЦП 121, невыписано -2 -1.7%, дата заполнения: Oct 30 13:56
3: 7-ки 122, ЕЦП 118, невыписано 4 3.3%, дата заполнения: Nov  1 08:47
4: 7-ки 74, ЕЦП 73, невыписано 1 1.4%, дата заполнения: Nov  1 08:44
0 107 ГОРДЕЕВ АРСЕНИЙ
1 107 ЗОРОВ ГЛЕБ
2 110 АЛЫШОВ ШАМАДДИН
3 107 МЕЛЬЦОВ СТЕПАН
4 107 ТЕПЛОУХОВ КИРИЛЛ
5 107 ХАЛИЛОВА

In [ ]:
counter=0
for p in pacients:
    if p['Исход'] not in ['101','105']:
        
        print(counter, p['Исход'], p['ФИО'])
        counter+=1

In [ ]:
for div in div_list:
    print (div['Отделение'][-1:]+': 7-ки', str(div['Выписано7'])+
           ', ЕЦП',str(div['ВыписаноЕЦП'])+
           ', невыписано',div['Выписано7'] - div['ВыписаноЕЦП'],
           "%.1f" %((div['Выписано7'] - div['ВыписаноЕЦП'])/div['Выписано7']*100)+'%'+
           ', дата заполнения:',time.ctime(os.path.getmtime(div['Файл']))[4:-8]
          )

In [ ]:
import time
for i in div_list:
    print('дата заполнения:',time.ctime(os.path.getmtime(i['Файл']))[4:-8])

In [ ]:
#-------------------------------------Посчитать по семёркам с 27 прошлого месяца по 27 текущего месяца - НЕДОДЕЛАНО

import requests, pickle, pandas, datetime, os, shutil, time

baseurl=os.getenv('BASE_URL')
api_person = baseurl+"/api/Person"
api_police = baseurl+"/api/Polis"
ecp_searchData = baseurl+"/?c=Search&m=searchData"
year=2025
month=3
month_before=month-1 if month > 1 else 12
sevens_total=[]
sevens_count,ecp_count,ecp_x=0,0,0

def login():
    print("Logining")
    headers = {
            "Login": os.getenv('ECP_USERNAME_1'),
            "Password": os.getenv('ECP_PASSWORD_1')
    }
    session.request("GET", f"{baseurl}/api/user/login", headers=headers)
    
def month_range(year,month):
    today=datetime.date.today()
    range_start=datetime.date(year,month,1).strftime("%d.%m.%Y")
    range_end=(datetime.date(year,month+1,1)-datetime.timedelta(days=1)).strftime("%d.%m.%Y")
    return range_start+" - " + range_end#"31.12.2024"#

def find_seven(year, month, path):
    file_list = os.listdir(path) #smbclient.listdir(path)
    for file in file_list:
        if (str(year)+'-'+str(month).zfill(2)) in file:
            sevens_filename=file
    return path+'\\'+sevens_filename

def write_data_to_sevens(sevens_filename, div_name):
    print('Start write_data_to_sevens',div_name, 'to', sevens_filename)
    sheets = pandas.ExcelFile(sevens_filename).sheet_names
    with pandas.ExcelWriter(sevens_filename, mode="a", engine="openpyxl",if_sheet_exists='overlay') as writer:
        print('Парсим',sevens_filename)
        for sheet in sheets:
            cur_sheet=writer.sheets[sheet]
            #отключить защиту
            cur_sheet.protection.password = '1111'
            cur_sheet.protection.disable()
            #определяемя за какой день текущий лист
            sheet_day_cell_value=pandas.read_excel(sevens_filename, sheet_name=sheet, skiprows=2, nrows=101, header=None).iloc[0][3]
            if type(sheet_day_cell_value)==datetime.datetime:
                sheet_day=(sheet_day_cell_value-datetime.timedelta(days=1)).strftime("%d.%m.%Y")
            else:
                sheet_day=(datetime.datetime.strptime(sheet_day, "%d.%m.%Y") + (datetime.timedelta(days=1))).strftime("%d.%m.%Y")
            #формируем и сортируем выписанных+переведённых по семёркам и выводим в 10 столбец:
            discharged=pandas.read_excel(sevens_filename, sheet_name=sheet, skiprows=8, usecols=[2]).dropna()
            transfered=pandas.read_excel(sevens_filename, sheet_name=sheet, skiprows=8, usecols=[6]).dropna()
            transfered.rename(columns={'Перевод\nв др. ЛПУ':'Выписано'}, inplace=True)
            discharged=pandas.concat([discharged,transfered])
            discharged['ДатаВыписки'] = sheet_day
            discharged['Отделение'] = div_name
            if len(discharged)>0:
                sevens_total.append(discharged.to_dict('records'))
            discharged.sort_values(by='Выписано').iloc[:,0].to_excel(writer, sheet_name=sheet, index=False, header=False, startrow=9, startcol=10)
            cur_sheet.cell(column=11, row=9, value=("Выписанные по семёркам" if len(discharged)!=0 else "Выписки в семёрках нет"))
            #формируем и сортируем выписанных+переведённых в ЕЦП из отделения в этот день и записываем в 11 столбец
            alive = pandas.DataFrame([pacient for pacient in pacients if pacient['ДатаВыписки']==sheet_day and pacient['Отделение']==div_name and pacient['Исход']!='105'])
            if len(alive)>0:
                alive.iloc[:,0].sort_values().to_excel(writer, sheet_name=sheet, index=False, startcol=11, startrow=9, header=False)
            cur_sheet.cell(column=12, row=9, value=("Выписанные в ЕЦП" if len(alive)!=0 else "Выписки в ЕЦП нет"))
            #формируем и сортируем умерших в ЕЦП за этим отделением в этот день и записываем в 12 столбец
            dead = pandas.DataFrame([pacient for pacient in pacients if pacient['ДатаВыписки']==sheet_day and pacient['Отделение']==div_name and pacient['Исход']=='105'])
            if len(dead)>0:
                dead.iloc[:,0].sort_values().to_excel(writer, sheet_name=sheet, index=False, startcol=12, startrow=9, header=False)
            cur_sheet.cell(column=13, row=9, value=("Трупы в ЕЦП" if len(dead)!=0 else "Трупов в ЕЦП нет"))
            if len(alive)==len(discharged) and len(alive)>0:
                #cur_sheet.sheet_properties.tabColor = '00FF00'
                #cur_sheet['e7'].style = Style(fill=PatternFill(patternType='solid', fgColor=Color('00FF00')))
                cur_sheet['e7'].style = 'Good'
            elif len(alive)==len(discharged) and len(alive)==0:
                #cur_sheet.sheet_properties.tabColor = 'FFFF00'
                cur_sheet['e7'].style = 'Neutral'
            else:
                #cur_sheet.sheet_properties.tabColor = 'FF0000'
                cur_sheet['e7'].style = 'Bad'
            cur_sheet['e8']=len(alive)
            #cur_sheet.protection.enable()
            
#------------------initialisation--------------------

div_list=[{'Отделение':'Инфекционное детское №1', 'Файл':find_seven(year,month,r'S:\06 - Отделения\1 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},
          {'Отделение':'Инфекционное №2', 'Файл':find_seven(year,month,r'S:\06 - Отделения\2 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},
          {'Отделение':'Инфекционное №3', 'Файл':find_seven(year,month,r'S:\06 - Отделения\3 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},
          {'Отделение':'Инфекционное №4', 'Файл':find_seven(year,month,r'S:\06 - Отделения\4 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},]

#------------------here-we-are-ready--------------------

pacients=[]

for div in div_list:
    shutil.copy2(div['Файл'],'.')
    write_data_to_sevens(div['Файл'].split('\\')[-1],div['Отделение'])
    #shutil.copy2(sevens_fullname.split('\\')[-1],sevens_fullname)

sevens_count=0
for day in sevens_total:
    for p in day:
        sevens_count+=1
        for div in div_list:
            if div['Отделение']==p['Отделение']:
                div['Выписано7']+=1

print(datetime.datetime.now())
print('ВСЕГО: по семёркам', sevens_count, 'выписано в ЕЦП', ecp_count, ', не выписано в ЕЦП', (sevens_count-(ecp_count)), ', трупы',ecp_x)
for div in div_list:
    print (div['Отделение'][-1:]+': 7-ки', str(div['Выписано7'])+
           ', ЕЦП',str(div['ВыписаноЕЦП'])+
           ', невыписано',div['Выписано7'] - div['ВыписаноЕЦП'],
           "%.1f" %((div['Выписано7'] - div['ВыписаноЕЦП'])/div['Выписано7']*100)+'%'+
           ', дата заполнения:',time.ctime(os.path.getmtime(div['Файл']))[4:-8]
          )

In [ ]:
#-------------------------------------Копируем семёрки

import requests, datetime, os, openpyxl
import calendar
from datetime import datetime

template_path='sevens template.xlsx' #<-шаблон для копирования
year=2025#<-----------------------------год
month=11 #<------------------------------инкрементируем месяц (это месяц за который надо сделать семёрки)
month_before=month-1 if month > 1 else 12
num_days = calendar.monthrange(year, month)[1] # Получаем количество дней в месяце
second_day_date = datetime(year, month, 2).strftime("%d.%m.%Y") # Второе число месяца в нужном формате
div_list=[{'Отделение':'Iотд', 'Файл':r'S:\06 - Отделения\1 отделение\Семёрки'},
          {'Отделение':'IIотд', 'Файл':r'S:\06 - Отделения\2 отделение\Семёрки'},
          {'Отделение':'IIIотд', 'Файл':r'S:\06 - Отделения\3 отделение\Семёрки'},
          {'Отделение':'IVотд', 'Файл':r'S:\06 - Отделения\4 отделение\Семёрки'},
          {'Отделение':'ДС', 'Файл':r'S:\06 - Отделения\Дневной стационар\Семёрки'},
          {'Отделение':'ОРиИТ', 'Файл':r'S:\06 - Отделения\ОРиИТ\Семёрки'},
          ]

def create_monthly_reports(template_path, year, month):
    for division in div_list:
        wb = openpyxl.load_workbook(template_path)
        while len(wb.worksheets) > num_days: # Удаляем лишнии листы
            wb.remove(wb.worksheets[-1])  
        first_sheet = wb.worksheets[0] # Получаем первый лист
        first_sheet['E2'] = division['Отделение']  # Название отделения
        first_sheet['D3'] = second_day_date        # Второе число месяца
        filename = f"{division['Файл']}\\семёрки {str(year)}-{str(month).zfill(2)} {division['Отделение']}.xlsx"
        if os.path.exists(filename):
            print(f"Ошибка! Файл '{filename}' уже существует.")
            continue
        wb.save(filename)
        print(f"Создан файл: {filename}")
create_monthly_reports(template_path, year, month)

Создан файл: S:\06 - Отделения\1 отделение\Семёрки\семёрки 2025-11 Iотд.xlsx
Создан файл: S:\06 - Отделения\2 отделение\Семёрки\семёрки 2025-11 IIотд.xlsx
Создан файл: S:\06 - Отделения\3 отделение\Семёрки\семёрки 2025-11 IIIотд.xlsx
Создан файл: S:\06 - Отделения\4 отделение\Семёрки\семёрки 2025-11 IVотд.xlsx
Создан файл: S:\06 - Отделения\Дневной стационар\Семёрки\семёрки 2025-11 ДС.xlsx
Создан файл: S:\06 - Отделения\ОРиИТ\Семёрки\семёрки 2025-11 ОРиИТ.xlsx


In [ ]:
import smbclient, json, openpyxl

from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.styles import Fill, Color

def damerau_levenshtein_distance(s1, s2):
    d = {}
    lenstr1 = len(s1)
    lenstr2 = len(s2)
    for i in range(-1,lenstr1+1):
        d[(i,-1)] = i+1
    for j in range(-1,lenstr2+1):
        d[(-1,j)] = j+1
 
    for i in range(lenstr1):
        for j in range(lenstr2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, # deletion
                           d[(i,j-1)] + 1, # insertion
                           d[(i-1,j-1)] + cost, # substitution
                          )
            if i and j and s1[i] == s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min(d[(i,j)], d[i-2,j-2] + 1) # transposition
    return d[lenstr1-1,lenstr2-1]

def similarity(s1,s2):
    dld=damerau_levenshtein_distance(s1, s2)
    if dld==0:
        return 1
    return 1-(dld)/len(s1)

def getSizeOfNestedList(listOfElem):
    count = 0
    for elem in listOfElem:
        if type(elem) == list:  
            count += getSizeOfNestedList(elem)
        else:
            count += 1    
    return count

In [ ]:
div_list=[{'Отделение':'Инфекционное детское №1', 'Файл':find_seven(year,month,r'S:\06 - Отделения\1 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},
          {'Отделение':'Инфекционное детское №2', 'Файл':find_seven(year,month,r'S:\06 - Отделения\2 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},
          {'Отделение':'Инфекционное №3 взрослое', 'Файл':find_seven(year,month,r'S:\06 - Отделения\3 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},
          {'Отделение':'Педиатрическое №4', 'Файл':find_seven(year,month,r'S:\06 - Отделения\4 отделение\Семёрки'), 'ВыписаноЕЦП':0, 'Выписано7':0},]

ecp_x=0

for pacient in kss_in_range_response['data']:
    if pacient['LeaveType_Code']=='105':
        print("105!!!!")
        continue
    if pacient['LeaveType_Code']!='104':
        print(pacient['LeaveType_Code'])
        pacients.append({'ФИО':pacient['Person_Surname']+" "+pacient['Person_Firname'],
                         'ДатаВыписки':pacient['EvnPS_disDate'],
                         'Отделение':pacient['LpuSection_Name'],
                         'Исход':pacient['LeaveType_Code']})
        if pacient['LeaveType_Code']!=105:
            ecp_count+=1
        else:
            ecp_x+=1
        for div in div_list:
            if div['Отделение']==pacient['LpuSection_Name'] and pacient['LeaveType_Code']!=105:
                div['ВыписаноЕЦП']+=1

print('пациентов в ЕЦП ' + str(len(pacients)))

# for div in div_list:
#     shutil.copy2(div['Файл'],'.')
#     write_data_to_sevens(div['Файл'].split('\\')[-1],div['Отделение'])
#     #shutil.copy2(sevens_fullname.split('\\')[-1],sevens_fullname)

# sevens_count=0
# for day in sevens_total:
#     for p in day:
#         sevens_count+=1
#         for div in div_list:
#             if div['Отделение']==p['Отделение']:
#                 div['Выписано7']+=1

print(datetime.datetime.now())
print('ВСЕГО: по семёркам', sevens_count, 'выписано в ЕЦП', ecp_count, ', не выписано в ЕЦП', (sevens_count-(ecp_count)), ', трупы',ecp_x)
for div in div_list:
    print (div['Отделение'],': выписано по семёркам', div['Выписано7'],
           ', выписано в ЕЦП',div['ВыписаноЕЦП'],
           ', невыписано',div['Выписано7'] - div['ВыписаноЕЦП'],
           # "%.1f" %((div['Выписано7'] - div['ВыписаноЕЦП'])/div['Выписано7']*100),'%'
          )